### 1. Preparar el entorno y las bibliotecas necesarias

In [1]:
# Instalación de las librerías necesarias
!pip install llama-cpp-python==0.1.78 numpy==1.23.4
!pip install huggingface_hub
!pip install sklearn


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
      --------------------------------------- 0.0/1.7 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.7 MB 1.2 MB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.7 MB 1.6 MB/s eta 0:00:01
     ------- -------------------------------- 0.3/1.7 MB 1.8 MB/s eta 0:00:01
     --------- ------------------------------ 0.4/1.7 MB 1.8 MB/s eta 0:00:01
     --------------- ------------------------ 0.7/1.7 MB 2.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 3.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 3.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 3.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 3.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 3.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 3.3 MB/s et


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

### 2. Importar librerías

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


c:\Users\diego\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 3. Cargar el modelo Llama desde Hugging Face

In [5]:
# Definir la ruta del modelo
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [6]:
# Descargar el modelo
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

c:\Users\diego\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\diego\.cache\huggingface\hub\models--TheBloke--Llama-2-13B-chat-GGML. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
# Cargar el modelo en la GPU
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # Usamos 2 núcleos de CPU
    n_batch=512,  # Tamaño del lote (ajustar según la VRAM)
    n_gpu_layers=32  # Capas en la GPU (ajustar según VRAM)
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


### 4. Crear un DataFrame con los documentos

In [8]:
# Datos de ejemplo relacionados con Ecuador
data = [
    {"file_name": "politica_1.txt", "content": "Ecuador ha tenido una historia política marcada por cambios de gobierno y movimientos sociales. El presidente actual ha centrado su agenda en mejorar la seguridad y la economía del país."},
    {"file_name": "geografia_1.txt", "content": "Ecuador es conocido por su biodiversidad, con el Amazonas, los Andes y las Islas Galápagos. Su clima varía de tropical a templado dependiendo de la región."},
    {"file_name": "cultura_1.txt", "content": "La cultura ecuatoriana está influenciada por sus raíces indígenas y coloniales. La música y la danza son parte fundamental de las celebraciones tradicionales, como el Inti Raymi."},
    {"file_name": "economia_1.txt", "content": "La economía de Ecuador depende en gran medida de las exportaciones de petróleo, pero también ha tenido un creciente desarrollo en el sector agrícola, especialmente el banano y el cacao."},
    {"file_name": "politica_2.txt", "content": "En los últimos años, el gobierno ecuatoriano ha trabajado para aumentar la inversión extranjera, mientras también aborda las tensiones sociales causadas por las reformas económicas."},
    {"file_name": "geografia_2.txt", "content": "Ecuador posee la mitad del mundo, atravesando la línea ecuatorial. Además, sus montañas, volcanes activos y selvas tropicales hacen del país un destino turístico popular."},
    {"file_name": "cultura_2.txt", "content": "Ecuador tiene una rica tradición artesanal, con comunidades que producen textiles, cerámica y joyería en varios pueblos a lo largo del país."},
    {"file_name": "economia_2.txt", "content": "A pesar de la dependencia del petróleo, Ecuador ha diversificado su economía mediante el desarrollo de su industria del turismo, especialmente en las Islas Galápagos."}
]

# Crear un DataFrame
df_ecuador = pd.DataFrame(data)


In [9]:
df_ecuador

,file_name,content
0,politica_1.txt,Ecuador ha tenido una historia política marcad...
1,geografia_1.txt,"Ecuador es conocido por su biodiversidad, con ..."
2,cultura_1.txt,La cultura ecuatoriana está influenciada por s...
3,economia_1.txt,La economía de Ecuador depende en gran medida ...
4,politica_2.txt,"En los últimos años, el gobierno ecuatoriano h..."
5,geografia_2.txt,"Ecuador posee la mitad del mundo, atravesando ..."
6,cultura_2.txt,"Ecuador tiene una rica tradición artesanal, co..."
7,economia_2.txt,"A pesar de la dependencia del petróleo, Ecuado..."


### 5. Implementar el sistema de Recuperación de Información (RI)

Usamos TF-IDF para calcular la similitud entre la query y los documentos, y recuperar los más relevantes.

In [10]:
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')

# Vectorizar los documentos y la consulta (query)
doc_vectors = vectorizer.fit_transform(df_ecuador['content'])
query = "¿Cómo es la economia de Ecuador?"
query_vector = vectorizer.transform([query])

# Calcular la similitud entre la consulta y los documentos
cosine_similarities = cosine_similarity(query_vector, doc_vectors)

# Obtener los índices de los documentos más relevantes
top_n = 3  # Número de documentos a recuperar
relevant_doc_indices = cosine_similarities.argsort()[0][-top_n:][::-1]

# Mostrar los documentos relevantes
relevant_docs = df_ecuador.iloc[relevant_doc_indices]
print("Documentos relevantes:")
for index, row in relevant_docs.iterrows():
    print(f"{row['file_name']}: {row['content']}\n")


Documentos relevantes:
geografia_1.txt: Ecuador es conocido por su biodiversidad, con el Amazonas, los Andes y las Islas Galápagos. Su clima varía de tropical a templado dependiendo de la región.

politica_1.txt: Ecuador ha tenido una historia política marcada por cambios de gobierno y movimientos sociales. El presidente actual ha centrado su agenda en mejorar la seguridad y la economía del país.

geografia_2.txt: Ecuador posee la mitad del mundo, atravesando la línea ecuatorial. Además, sus montañas, volcanes activos y selvas tropicales hacen del país un destino turístico popular.



### 6. Generación Aumentada por Recuperación (RAG)

In [11]:
# Obtener el contexto de los documentos relevantes
context = "\n".join(relevant_docs['content'])

# Definir el prompt que incluirá la consulta y el contexto
prompt_template = f'''SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: {query}

CONTEXT: {context}

ASSISTANT:
'''

# Generar la respuesta con Llama
response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.3, top_p=0.95, repeat_penalty=1.2, top_k=150, echo=True)

# Imprimir la respuesta generada
print(response["choices"][0]["text"])


SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: ¿Cómo es la economia de Ecuador?

CONTEXT: Ecuador es conocido por su biodiversidad, con el Amazonas, los Andes y las Islas Galápagos. Su clima varía de tropical a templado dependiendo de la región.
Ecuador ha tenido una historia política marcada por cambios de gobierno y movimientos sociales. El presidente actual ha centrado su agenda en mejorar la seguridad y la economía del país.
Ecuador posee la mitad del mundo, atravesando la línea ecuatorial. Además, sus montañas, volcanes activos y selvas tropicales hacen del país un destino turístico popular.

ASSISTANT:
¡Hola! ¡Claro que sí! La economía de Ecuador es diversa y en constante crecimiento. A continuación, te brindo algunos puntos clave sobre la situación actual:

1. Agricultura: El sector agrícola es una de las principales contribuidoras a la economía ecuatoriana. Los productos más destacados son el cacao, el coffee, el banano y los frutas tropicales. La

### Otro ejemplo

In [14]:
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')

# Vectorizar los documentos y la consulta (query)
doc_vectors = vectorizer.fit_transform(df_ecuador['content'])
query = "¿Hay cacao en Ecuador?"
query_vector = vectorizer.transform([query])

# Calcular la similitud entre la consulta y los documentos
cosine_similarities = cosine_similarity(query_vector, doc_vectors)

# Obtener los índices de los documentos más relevantes
top_n = 3  # Número de documentos a recuperar
relevant_doc_indices = cosine_similarities.argsort()[0][-top_n:][::-1]

# Mostrar los documentos relevantes
relevant_docs = df_ecuador.iloc[relevant_doc_indices]
print("Documentos relevantes:")
for index, row in relevant_docs.iterrows():
    print(f"{row['file_name']}: {row['content']}\n")


Documentos relevantes:
economia_1.txt: La economía de Ecuador depende en gran medida de las exportaciones de petróleo, pero también ha tenido un creciente desarrollo en el sector agrícola, especialmente el banano y el cacao.

economia_2.txt: A pesar de la dependencia del petróleo, Ecuador ha diversificado su economía mediante el desarrollo de su industria del turismo, especialmente en las Islas Galápagos.

cultura_2.txt: Ecuador tiene una rica tradición artesanal, con comunidades que producen textiles, cerámica y joyería en varios pueblos a lo largo del país.



In [15]:
# Obtener el contexto de los documentos relevantes
context = "\n".join(relevant_docs['content'])

# Definir el prompt que incluirá la consulta y el contexto
prompt_template = f'''SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: {query}

CONTEXT: {context}

ASSISTANT:
'''

# Generar la respuesta con Llama
response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.3, top_p=0.95, repeat_penalty=1.2, top_k=150, echo=True)

# Imprimir la respuesta generada
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SYSTEM: Eres una asistente personal que busca ayudar lo mejor posible.

USER: ¿Hay cacao en Ecuador?

CONTEXT: La economía de Ecuador depende en gran medida de las exportaciones de petróleo, pero también ha tenido un creciente desarrollo en el sector agrícola, especialmente el banano y el cacao.
A pesar de la dependencia del petróleo, Ecuador ha diversificado su economía mediante el desarrollo de su industria del turismo, especialmente en las Islas Galápagos.
Ecuador tiene una rica tradición artesanal, con comunidades que producen textiles, cerámica y joyería en varios pueblos a lo largo del país.

ASSISTANT:
¡Claro! Ecuador es famoso por su producción de cacao y es uno de los mayores productores de América Latina. La región más conocida para la producción de cacao en Ecuador es el valle del Guayas, específicamente en las provincias de Guayaquil y Cañar.
La industria del cacao en Ecuador se ha desarrollado gracias a su clima y suelo ideales para el cultivo, así como por la tecnología y